Téléchargement des archives zip

In [1]:
import requests
import os
import zipfile
import os
import re
import shutil

def creer_repertoires_annuels(chemin_base):
    """
    Crée des répertoires de 2013 à 2024 avec des sous-répertoires pour chaque mois.

    :param chemin_base: Chemin de base où les répertoires seront créés.
    """
    for annee in range(2013, 2025):
        repertoire_annee = os.path.join(chemin_base, str(annee))
        os.makedirs(repertoire_annee, exist_ok=True)
        
        for mois in range(1, 13):
            repertoire_mois = os.path.join(repertoire_annee, f'{mois:02}')
            os.makedirs(repertoire_mois, exist_ok=True)


def telecharger_fichiers(zip_urls, repertoire_destination):
    """
    Télécharge tous les fichiers .zip depuis le dictionnaire zip_urls et les stocke dans le répertoire donné.

    :param zip_urls: Dictionnaire contenant les noms de fichiers et les liens de téléchargement.
    :param repertoire_destination: Chemin du répertoire où les fichiers seront stockés.
    """
    # Vérifie et crée le répertoire de destination s'il n'existe pas
    if not os.path.exists(repertoire_destination):
        os.makedirs(repertoire_destination)

    for nom_fichier, url in zip_urls.items():
        chemin_fichier = os.path.join(repertoire_destination, nom_fichier)
        try:
            print(f'Téléchargement de {nom_fichier} en cours...')
            print("PATIENTER SVP...\n\n")
            response = requests.get(url, stream=True)
            response.raise_for_status()  # Vérifie si la requête a réussi

            # Téléchargement du fichier par morceaux
            with open(chemin_fichier, 'wb') as fichier:
                for chunk in response.iter_content(chunk_size=8192):
                    fichier.write(chunk)
            
            print(f'{nom_fichier} a été téléchargé et enregistré dans {repertoire_destination}')
        except requests.exceptions.RequestException as e:
            print(f"Erreur lors du téléchargement de {nom_fichier} : {e}")
    print("\n!!!!!!!!!!!!!!!!!!!!LES TELECHARGEMENTS DES FICHIERS .ZIP SONT TERMINES!!!!!!!!!!!!!!!!\n")



def supprimer_fichiers_caches(racine):
    """
    Parcourt tous les répertoires et sous-répertoires pour supprimer tous les fichiers cachés.
    
    :param racine: Chemin du répertoire racine à partir duquel la suppression doit être effectuée.
    """
    for root, dirs, files in os.walk(racine):
        for fichier in files:
            if fichier.startswith('.'):
                chemin_complet = os.path.join(root, fichier)
                try:
                    os.remove(chemin_complet)
                    print(f'Supprimé: {chemin_complet}')
                except Exception as e:
                    print(f'Erreur lors de la suppression de {chemin_complet}: {e}')
    
    print("!!!!!!!!!!ORGANISATION DES FICHIERS .CSV EN FONCTION DE L'ANNÉE ET DES MOIS EST TERMINÉ!!!!!!!")


def organiser_fichiers_zip(base_path):
    """
    Parcourt tous les fichiers .zip dans un répertoire donné, extrait les fichiers et les organise
    dans les répertoires de destination basés sur l'année et le mois trouvés dans les noms de fichiers.

    :param base_path: Chemin de base des répertoires organisés par année et mois.
    """
    # Parcourt tous les fichiers .zip dans le répertoire donné
    for fichier_zip in os.listdir(base_path):
        if fichier_zip.endswith('.zip'):
            chemin_fichier_zip = os.path.join(base_path, fichier_zip)
            with zipfile.ZipFile(chemin_fichier_zip, 'r') as zip_ref:
                for fichier in zip_ref.namelist():
                    # Recherche de l'année et du mois dans le nom du fichier
                    match = re.search(r'(\d{4})(\d{2})', fichier)
                    if match:
                        annee = match.group(1)
                        mois = match.group(2)
                        # Définir le répertoire de destination
                        repertoire_destination = os.path.join(base_path, annee, mois)
                        if not os.path.exists(repertoire_destination):
                            os.makedirs(repertoire_destination)
                        chemin_fichier_destination = os.path.join(repertoire_destination, os.path.basename(fichier))
                        # Vérifier si le fichier existe déjà pour éviter les duplications
                        if not os.path.exists(chemin_fichier_destination):
                            with zip_ref.open(fichier) as source, open(chemin_fichier_destination, 'wb') as target:
                                shutil.copyfileobj(source, target)
                            print(f'Fichier {fichier} extrait et copié dans {repertoire_destination}')
                        else:
                            print(f'Fichier {fichier} déjà présent dans {repertoire_destination}, copie évitée.')
            # Supprimer le fichier .zip après extraction
            os.remove(chemin_fichier_zip)
            print(f'Fichier .zip {chemin_fichier_zip} supprimé après extraction.\n')
            print("#########################PATIENTER SVP...#######################\n\n")


zip_urls = {
    'trip_data_2013.zip': "https://s3.amazonaws.com/tripdata/2013-citibike-tripdata.zip",
    'trip_data_2014.zip': "https://s3.amazonaws.com/tripdata/2014-citibike-tripdata.zip",
    'trip_data_2015.zip': "https://s3.amazonaws.com/tripdata/2015-citibike-tripdata.zip",
    'trip_data_2016.zip': "https://s3.amazonaws.com/tripdata/2016-citibike-tripdata.zip",
    'trip_data_2017.zip': "https://s3.amazonaws.com/tripdata/2017-citibike-tripdata.zip",
    'trip_data_2018.zip': "https://s3.amazonaws.com/tripdata/2018-citibike-tripdata.zip",
    'trip_data_2019.zip': "https://s3.amazonaws.com/tripdata/2019-citibike-tripdata.zip",
    'trip_data_2020.zip': "https://s3.amazonaws.com/tripdata/2020-citibike-tripdata.zip",
    'trip_data_2021.zip': "https://s3.amazonaws.com/tripdata/2021-citibike-tripdata.zip",
    'trip_data_2022.zip': "https://s3.amazonaws.com/tripdata/2022-citibike-tripdata.zip",
    'trip_data_2023.zip': "https://s3.amazonaws.com/tripdata/2023-citibike-tripdata.zip",


    '202401-citibike-tripdata.csv.zip': "https://s3.amazonaws.com/tripdata/202401-citibike-tripdata.csv.zip",
    '202402-citibike-tripdata.csv.zip': "https://s3.amazonaws.com/tripdata/202402-citibike-tripdata.csv.zip",
    '202403-citibike-tripdata.csv.zip': "https://s3.amazonaws.com/tripdata/202403-citibike-tripdata.csv.zip",
    '202404-citibike-tripdata.csv.zip': "https://s3.amazonaws.com/tripdata/202404-citibike-tripdata.csv.zip",


    'JC-201509-citibike-tripdata.csv.zip' : "https://s3.amazonaws.com/tripdata/JC-201509-citibike-tripdata.csv.zip",
    'JC-201510-citibike-tripdata.csv.zip' : "https://s3.amazonaws.com/tripdata/JC-201510-citibike-tripdata.csv.zip",
    'JC-201511-citibike-tripdata.csv.zip' : "https://s3.amazonaws.com/tripdata/JC-201511-citibike-tripdata.csv.zip",
    'JC-201512-citibike-tripdata.csv.zip' : "https://s3.amazonaws.com/tripdata/JC-201512-citibike-tripdata.csv.zip",


    'JC-201601-citibike-tripdata.csv.zip' : "https://s3.amazonaws.com/tripdata/JC-201601-citibike-tripdata.csv.zip",
    'JC-201602-citibike-tripdata.csv.zip' : "https://s3.amazonaws.com/tripdata/JC-201602-citibike-tripdata.csv.zip",
    'JC-201603-citibike-tripdata.csv.zip' : "https://s3.amazonaws.com/tripdata/JC-201603-citibike-tripdata.csv.zip",
    'JC-201604-citibike-tripdata.csv.zip' : "https://s3.amazonaws.com/tripdata/JC-201604-citibike-tripdata.csv.zip",
    'JC-201605-citibike-tripdata.csv.zip' : "https://s3.amazonaws.com/tripdata/JC-201605-citibike-tripdata.csv.zip",
    'JC-201606-citibike-tripdata.csv.zip' : "https://s3.amazonaws.com/tripdata/JC-201606-citibike-tripdata.csv.zip",
    'JC-201607-citibike-tripdata.csv.zip' : "https://s3.amazonaws.com/tripdata/JC-201607-citibike-tripdata.csv.zip",
    'JC-201608-citibike-tripdata.csv.zip' : "https://s3.amazonaws.com/tripdata/JC-201608-citibike-tripdata.csv.zip",
    'JC-201609-citibike-tripdata.csv.zip' : "https://s3.amazonaws.com/tripdata/JC-201609-citibike-tripdata.csv.zip",
    'JC-201610-citibike-tripdata.csv.zip' : "https://s3.amazonaws.com/tripdata/JC-201610-citibike-tripdata.csv.zip",
    'JC-201611-citibike-tripdata.csv.zip' : "https://s3.amazonaws.com/tripdata/JC-201611-citibike-tripdata.csv.zip",
    'JC-201612-citibike-tripdata.csv.zip' : "https://s3.amazonaws.com/tripdata/JC-201612-citibike-tripdata.csv.zip",
    
    
    'JC-201701-citibike-tripdata.csv.zip' : "https://s3.amazonaws.com/tripdata/JC-201701-citibike-tripdata.csv.zip",
    'JC-201702-citibike-tripdata.csv.zip' : "https://s3.amazonaws.com/tripdata/JC-201702-citibike-tripdata.csv.zip",
    'JC-201703-citibike-tripdata.csv.zip' : "https://s3.amazonaws.com/tripdata/JC-201703-citibike-tripdata.csv.zip",
    'JC-201704-citibike-tripdata.csv.zip' : "https://s3.amazonaws.com/tripdata/JC-201704-citibike-tripdata.csv.zip",
    'JC-201705-citibike-tripdata.csv.zip' : "https://s3.amazonaws.com/tripdata/JC-201705-citibike-tripdata.csv.zip",
    'JC-201706-citibike-tripdata.csv.zip' : "https://s3.amazonaws.com/tripdata/JC-201706-citibike-tripdata.csv.zip",
    'JC-201707-citibike-tripdata.csv.zip' : "https://s3.amazonaws.com/tripdata/JC-201707-citibike-tripdata.csv.zip",
    'JC-201708-citibike-tripdata.csv.zip' : "https://s3.amazonaws.com/tripdata/JC-201708%20citibike-tripdata.csv.zip",
    'JC-201709-citibike-tripdata.csv.zip' : "https://s3.amazonaws.com/tripdata/JC-201709-citibike-tripdata.csv.zip",
    'JC-201710-citibike-tripdata.csv.zip' : "https://s3.amazonaws.com/tripdata/JC-201710-citibike-tripdata.csv.zip",
    'JC-201711-citibike-tripdata.csv.zip' : "https://s3.amazonaws.com/tripdata/JC-201711-citibike-tripdata.csv.zip",
    'JC-201712-citibike-tripdata.csv.zip' : "https://s3.amazonaws.com/tripdata/JC-201712-citibike-tripdata.csv.zip",
    
    
    'JC-201801-citibike-tripdata.csv.zip' : "https://s3.amazonaws.com/tripdata/JC-201801-citibike-tripdata.csv.zip",
    'JC-201802-citibike-tripdata.csv.zip' : "https://s3.amazonaws.com/tripdata/JC-201802-citibike-tripdata.csv.zip",
    'JC-201803-citibike-tripdata.csv.zip' : "https://s3.amazonaws.com/tripdata/JC-201803-citibike-tripdata.csv.zip",
    'JC-201804-citibike-tripdata.csv.zip' : "https://s3.amazonaws.com/tripdata/JC-201804-citibike-tripdata.csv.zip",
    'JC-201805-citibike-tripdata.csv.zip' : "https://s3.amazonaws.com/tripdata/JC-201805-citibike-tripdata.csv.zip",
    'JC-201806-citibike-tripdata.csv.zip' : "https://s3.amazonaws.com/tripdata/JC-201806-citibike-tripdata.csv.zip",
    'JC-201807-citibike-tripdata.csv.zip' : "https://s3.amazonaws.com/tripdata/JC-201807-citibike-tripdata.csv.zip",
    'JC-201808-citibike-tripdata.csv.zip' : "https://s3.amazonaws.com/tripdata/JC-201808-citibike-tripdata.csv.zip",
    'JC-201809-citibike-tripdata.csv.zip' : "https://s3.amazonaws.com/tripdata/JC-201809-citibike-tripdata.csv.zip",
    'JC-201810-citibike-tripdata.csv.zip' : "https://s3.amazonaws.com/tripdata/JC-201810-citibike-tripdata.csv.zip",
    'JC-201811-citibike-tripdata.csv.zip' : "https://s3.amazonaws.com/tripdata/JC-201811-citibike-tripdata.csv.zip",
    'JC-201812-citibike-tripdata.csv.zip' : "https://s3.amazonaws.com/tripdata/JC-201812-citibike-tripdata.csv.zip",
    
    
    'JC-201901-citibike-tripdata.csv.zip' : "https://s3.amazonaws.com/tripdata/JC-201901-citibike-tripdata.csv.zip",
    'JC-201902-citibike-tripdata.csv.zip' : "https://s3.amazonaws.com/tripdata/JC-201902-citibike-tripdata.csv.zip",
    'JC-201903-citibike-tripdata.csv.zip' : "https://s3.amazonaws.com/tripdata/JC-201903-citibike-tripdata.csv.zip",
    'JC-201904-citibike-tripdata.csv.zip' : "https://s3.amazonaws.com/tripdata/JC-201904-citibike-tripdata.csv.zip",
    'JC-201905-citibike-tripdata.csv.zip' : "https://s3.amazonaws.com/tripdata/JC-201905-citibike-tripdata.csv.zip",
    'JC-201906-citibike-tripdata.csv.zip' : "https://s3.amazonaws.com/tripdata/JC-201906-citibike-tripdata.csv.zip",
    'JC-201907-citibike-tripdata.csv.zip' : "https://s3.amazonaws.com/tripdata/JC-201907-citibike-tripdata.csv.zip",
    'JC-201908-citibike-tripdata.csv.zip' : "https://s3.amazonaws.com/tripdata/JC-201908-citibike-tripdata.csv.zip",
    'JC-201909-citibike-tripdata.csv.zip' : "https://s3.amazonaws.com/tripdata/JC-201909-citibike-tripdata.csv.zip",
    'JC-201910-citibike-tripdata.csv.zip' : "https://s3.amazonaws.com/tripdata/JC-201910-citibike-tripdata.csv.zip",
    'JC-201911-citibike-tripdata.csv.zip' : "https://s3.amazonaws.com/tripdata/JC-201911-citibike-tripdata.csv.zip",
    'JC-201912-citibike-tripdata.csv.zip' : "https://s3.amazonaws.com/tripdata/JC-201912-citibike-tripdata.csv.zip",
    
    
    'JC-202001-citibike-tripdata.csv.zip' : "https://s3.amazonaws.com/tripdata/JC-202001-citibike-tripdata.csv.zip",
    'JC-202002-citibike-tripdata.csv.zip' : "https://s3.amazonaws.com/tripdata/JC-202002-citibike-tripdata.csv.zip",
    'JC-202003-citibike-tripdata.csv.zip' : "https://s3.amazonaws.com/tripdata/JC-202003-citibike-tripdata.csv.zip",
    'JC-202004-citibike-tripdata.csv.zip' : "https://s3.amazonaws.com/tripdata/JC-202004-citibike-tripdata.csv.zip",
    'JC-202005-citibike-tripdata.csv.zip' : "https://s3.amazonaws.com/tripdata/JC-202005-citibike-tripdata.csv.zip",
    'JC-202006-citibike-tripdata.csv.zip' : "https://s3.amazonaws.com/tripdata/JC-202006-citibike-tripdata.csv.zip",
    'JC-202007-citibike-tripdata.csv.zip' : "https://s3.amazonaws.com/tripdata/JC-202007-citibike-tripdata.csv.zip",
    'JC-202008-citibike-tripdata.csv.zip' : "https://s3.amazonaws.com/tripdata/JC-202008-citibike-tripdata.csv.zip",
    'JC-202009-citibike-tripdata.csv.zip' : "https://s3.amazonaws.com/tripdata/JC-202009-citibike-tripdata.csv.zip",
    'JC-202010-citibike-tripdata.csv.zip' : "https://s3.amazonaws.com/tripdata/JC-202010-citibike-tripdata.csv.zip",
    'JC-202011-citibike-tripdata.csv.zip' : "https://s3.amazonaws.com/tripdata/JC-202011-citibike-tripdata.csv.zip",
    'JC-202012-citibike-tripdata.csv.zip' : "https://s3.amazonaws.com/tripdata/JC-202012-citibike-tripdata.csv.zip",


    'JC-202101-citibike-tripdata.csv.zip' : "https://s3.amazonaws.com/tripdata/JC-202101-citibike-tripdata.csv.zip",
    'JC-202102-citibike-tripdata.csv.zip' : "https://s3.amazonaws.com/tripdata/JC-202102-citibike-tripdata.csv.zip",
    'JC-202103-citibike-tripdata.csv.zip' : "https://s3.amazonaws.com/tripdata/JC-202103-citibike-tripdata.csv.zip",
    'JC-202104-citibike-tripdata.csv.zip' : "https://s3.amazonaws.com/tripdata/JC-202104-citibike-tripdata.csv.zip",
    'JC-202105-citibike-tripdata.csv.zip' : "https://s3.amazonaws.com/tripdata/JC-202105-citibike-tripdata.csv.zip",
    'JC-202106-citibike-tripdata.csv.zip' : "https://s3.amazonaws.com/tripdata/JC-202106-citibike-tripdata.csv.zip",
    'JC-202107-citibike-tripdata.csv.zip' : "https://s3.amazonaws.com/tripdata/JC-202107-citibike-tripdata.csv.zip",
    'JC-202108-citibike-tripdata.csv.zip' : "https://s3.amazonaws.com/tripdata/JC-202108-citibike-tripdata.csv.zip",
    'JC-202109-citibike-tripdata.csv.zip' : "https://s3.amazonaws.com/tripdata/JC-202109-citibike-tripdata.csv.zip",
    'JC-202110-citibike-tripdata.csv.zip' : "https://s3.amazonaws.com/tripdata/JC-202110-citibike-tripdata.csv.zip",
    'JC-202111-citibike-tripdata.csv.zip' : "https://s3.amazonaws.com/tripdata/JC-202111-citibike-tripdata.csv.zip",
    'JC-202112-citibike-tripdata.csv.zip' : "https://s3.amazonaws.com/tripdata/JC-202112-citibike-tripdata.csv.zip",


    'JC-202201-citibike-tripdata.csv.zip' : "https://s3.amazonaws.com/tripdata/JC-202201-citibike-tripdata.csv.zip",
    'JC-202202-citibike-tripdata.csv.zip' : "https://s3.amazonaws.com/tripdata/JC-202202-citibike-tripdata.csv.zip",
    'JC-202203-citibike-tripdata.csv.zip' : "https://s3.amazonaws.com/tripdata/JC-202203-citibike-tripdata.csv.zip",
    'JC-202204-citibike-tripdata.csv.zip' : "https://s3.amazonaws.com/tripdata/JC-202204-citibike-tripdata.csv.zip",
    'JC-202205-citibike-tripdata.csv.zip' : "https://s3.amazonaws.com/tripdata/JC-202205-citibike-tripdata.csv.zip",
    'JC-202206-citibike-tripdata.csv.zip' : "https://s3.amazonaws.com/tripdata/JC-202206-citibike-tripdata.csv.zip",
    'JC-202207-citibike-tripdata.csv.zip' : "https://s3.amazonaws.com/tripdata/JC-202207-citbike-tripdata.csv.zip",
    'JC-202208-citibike-tripdata.csv.zip' : "https://s3.amazonaws.com/tripdata/JC-202208-citibike-tripdata.csv.zip",
    'JC-202209-citibike-tripdata.csv.zip' : "https://s3.amazonaws.com/tripdata/JC-202209-citibike-tripdata.csv.zip",
    'JC-202210-citibike-tripdata.csv.zip' : "https://s3.amazonaws.com/tripdata/JC-202210-citibike-tripdata.csv.zip",
    'JC-202211-citibike-tripdata.csv.zip' : "https://s3.amazonaws.com/tripdata/JC-202211-citibike-tripdata.csv.zip",
    'JC-202212-citibike-tripdata.csv.zip' : "https://s3.amazonaws.com/tripdata/JC-202212-citibike-tripdata.csv.zip",


    'JC-202301-citibike-tripdata.csv.zip' : "https://s3.amazonaws.com/tripdata/JC-202301-citibike-tripdata.csv.zip",
    'JC-202302-citibike-tripdata.csv.zip' : "https://s3.amazonaws.com/tripdata/JC-202302-citibike-tripdata.csv.zip",
    'JC-202303-citibike-tripdata.csv.zip' : "https://s3.amazonaws.com/tripdata/JC-202303-citibike-tripdata.csv.zip",
    'JC-202304-citibike-tripdata.csv.zip' : "https://s3.amazonaws.com/tripdata/JC-202304-citibike-tripdata.csv.zip",
    'JC-202305-citibike-tripdata.csv.zip' : "https://s3.amazonaws.com/tripdata/JC-202305-citibike-tripdata.csv.zip",
    'JC-202306-citibike-tripdata.csv.zip' : "https://s3.amazonaws.com/tripdata/JC-202306-citibike-tripdata.csv.zip",
    'JC-202307-citibike-tripdata.csv.zip' : "https://s3.amazonaws.com/tripdata/JC-202307-citibike-tripdata.csv.zip",
    'JC-202308-citibike-tripdata.csv.zip' : "https://s3.amazonaws.com/tripdata/JC-202308-citibike-tripdata.csv.zip",
    'JC-202309-citibike-tripdata.csv.zip' : "https://s3.amazonaws.com/tripdata/JC-202309-citibike-tripdata.csv.zip",
    'JC-202310-citibike-tripdata.csv.zip' : "https://s3.amazonaws.com/tripdata/JC-202310-citibike-tripdata.csv.zip",
    'JC-202311-citibike-tripdata.csv.zip' : "https://s3.amazonaws.com/tripdata/JC-202311-citibike-tripdata.csv.zip",
    'JC-202312-citibike-tripdata.csv.zip' : "https://s3.amazonaws.com/tripdata/JC-202312-citibike-tripdata.csv.zip",


    'JC-202401-citibike-tripdata.csv.zip' : "https://s3.amazonaws.com/tripdata/JC-202401-citibike-tripdata.csv.zip",
    'JC-202402-citibike-tripdata.csv.zip' : "https://s3.amazonaws.com/tripdata/JC-202402-citibike-tripdata.csv.zip",
    'JC-202403-citibike-tripdata.csv.zip' : "https://s3.amazonaws.com/tripdata/JC-202403-citibike-tripdata.csv.zip",
    'JC-202404-citibike-tripdata.csv.zip' : "https://s3.amazonaws.com/tripdata/JC-202404-citibike-tripdata.csv.zip"
}

base_path  = './csv_data/'  # Remplacez ceci par le chemin de votre répertoire de téléchargement
creer_repertoires_annuels(base_path)
telecharger_fichiers(zip_urls, base_path)
organiser_fichiers_zip(base_path)
supprimer_fichiers_caches(base_path)

Téléchargement de trip_data_2013.zip en cours...
PATIENTER SVP...


trip_data_2013.zip a été téléchargé et enregistré dans ./csv_data/
Téléchargement de trip_data_2014.zip en cours...
PATIENTER SVP...


trip_data_2014.zip a été téléchargé et enregistré dans ./csv_data/
Téléchargement de trip_data_2015.zip en cours...
PATIENTER SVP...


trip_data_2015.zip a été téléchargé et enregistré dans ./csv_data/
Téléchargement de trip_data_2016.zip en cours...
PATIENTER SVP...


trip_data_2016.zip a été téléchargé et enregistré dans ./csv_data/
Téléchargement de trip_data_2017.zip en cours...
PATIENTER SVP...


trip_data_2017.zip a été téléchargé et enregistré dans ./csv_data/
Téléchargement de trip_data_2018.zip en cours...
PATIENTER SVP...


trip_data_2018.zip a été téléchargé et enregistré dans ./csv_data/
Téléchargement de trip_data_2019.zip en cours...
PATIENTER SVP...


trip_data_2019.zip a été téléchargé et enregistré dans ./csv_data/
Téléchargement de trip_data_2020.zip en cours...
PATIEN

Conversion des fichiers CSV en un format compatible avec Spark

Sauvegarde des données dans un format de Big Data (parquet/ORC)

Organisation des données en un schéma en étoile

Conseils : 
- N'oubliez pas de demander à Spark/Dask d'utiliser une part substantielle de la mémoire et des ressources de votre ordinateur.
- N'oubliez pas de spécifier une colonne de partitionnement et un nombre de partitions lors de la création des fichiers parquet.


Questions à se poser : 
- Quel est le StorageLevel du dataframe après la lecture des fichiers csv ?
- Quel est le nombre de partitions du dataframe ?
- Est-il possible de régler ce nombre au moment du chargement ?
- Pourquoi voudrions-nous modifier le nombre de partitions lors de la création des fichiers parquet ?

Analyse des données avant et après Covid : 

Analysez (au moins) une année de données avant Covid et une année après Covid.

Calculez au moins les éléments suivants et produisez des graphiques pertinents :
1. Répartition des distances des trajets pour chaque jour de la semaine
2. Compter le nombre de trajets pour chaque couple de lieux de départ/arrivée
3. Répartition des distances des trajets par genre (lorsque disponible)
4. Répartition des distances des trajets par tranches d'âge (15-24, 25-44, 45-54, 55-64, 65+ ans) (lorsque disponible)
5. Répartition des distances des trajets pour différents types de vélos (lorsque disponible)



Conseils pour les graphiques : 
Si vous souhaitez créer des graphiques attrayants, vous pouvez utiliser matplotlib + seaborn, plotly ou altair pour créer des graphiques interactifs, mais vous pouvez utiliser ce que vous voulez.

Évaluation des saisonnalités et analyse des séries temporelles

Calculez et tracez les séries temporelles suivantes indexées par jour de la semaine et heure de la journée :
1. Le nombre de départs/arrivées
2. La distance moyenne
3. La durée moyenne des trajets
4. Le nombre moyen de trajets en cours

Suivi de l'exécution des travaux

1. Utilisez la méthode `explain` ou consultez l'interface utilisateur de Spark pour analyser le travail. Vous devriez pouvoir évaluer :
   - Plan logique analysé
   - Plan logique optimisé
   - Plan physique

2. Les plans logique analysé et optimisé diffèrent-ils ? Repérez les différences, le cas échéant. Comment une SGBDR procéderait-elle à une telle requête ?

3. Comment le plan physique diffère-t-il du plan logique optimisé ? Quels sont les mots-clés que vous n'attendez pas dans une SGBDR ? Quelle est leur signification ?

4. Inspectez les étapes sur l'interface utilisateur de Spark. Combien d'étapes sont nécessaires pour compléter le travail Spark ? Quels sont les rôles de `HashAggregate` et `Exchange hashpartitioning` ?

5. Le plan physique effectue-t-il des opérations de shuffle ? Si oui, combien ?
6. Qu'est-ce que les tâches par rapport aux étapes (en langage Spark) ? Combien de tâches composent vos étapes ?

Plongée dans les problèmes d'information spatiale


**Informations géographiques**

**Choisissez une année**
Pour cela, vous devrez trouver des outils pour afficher des cartes et créer des cartes choroplèthes. Nous vous laissons rechercher et trouver les outils pertinents pour cela.

1. Créez une carte de chaleur indexée par couple de stations où la couleur est fonction du nombre de trajets d'une station à une autre.
2. Créez une carte de chaleur interactive avec un curseur permettant à l'utilisateur de sélectionner une heure de la journée et où la couleur est fonction du nombre de trajets d'une station à une autre.

Vous pourriez avoir besoin des éléments suivants pour travailler avec des coordonnées GPS et tracer facilement les choses.